# Algoritmos de Optimización.
## Samuel De Dios

## Programación lineal.

Importar libreria.

In [7]:
!pip install pulp
from pulp import *


[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Ejemplo min.

Se usa de ejemplo este ejercicio.

\begin{align*}
\text{Minimizar }&19x_{1}+57x_{2} \\
\text{Restric. }& x_{1} + 2x_{2} \geq 10\\
& x_{1} + 4x_{2} \geq 24\\
& 3x_{1} + 2x_{2} \geq 20\\
& 5x_{1} + x_{2} \geq 25\\
\end{align*}

Se define un objeto problema, luego se pone nombre y lo que se quiere hacer (minimizar o maximizar), luego se definen las variables del problema.

In [8]:
problema = LpProblem("proLineal", LpMinimize)
x1 = LpVariable("x1", lowBound = 0)
x2 = LpVariable("x2", lowBound = 0)

Se agrega primero la función objetivo y despues las restricciones.

In [9]:
problema += 19*x1 + 57*x2
problema += x1 + 2*x2 >= 10
problema += x1 + 4*x2 >= 24
problema += 3*x1 + 2*x2 >= 20
problema += 5*x1 + x2 >= 25

Se solucciona el problema.

In [10]:
problema.solve()

print(f"x1 = {x1.varValue} y x2 = {x2.varValue}, con valor optimo {value(problema.objective)}")

x1 = 4.0 y x2 = 5.0, con valor optimo 361.0


### Ejemplo max.

\begin{align*}
\text{Minimizar }&10x_{1} + 24x_{2} + 20x_{3} + 35x_{4} \\
\text{Restric. }& x_{1} + x_{2} + 3x_{3} + 5x_{4} \leq 19\\
& 2x_{1} + 4x_{2} + 2x_{3} + x_{4} \leq 57\\
\end{align*}

Definicion del problema.

In [11]:
problema = LpProblem("prolineal", LpMaximize)
x1 = LpVariable("x1", lowBound = 0)
x2 = LpVariable("x2", lowBound = 0)
x3 = LpVariable("x3", lowBound = 0)
x4 = LpVariable("x4", lowBound = 0)

problema += 10*x1 + 24*x2 + 20*x3 + 35*x4
problema += x1 + x2 + 3*x3 + 5*x4 <= 19
problema += 2*x1 + 4*x2 + 2*x3 + x4 <= 57

In [12]:
problema.solve()

print(f"x1 = {x1.varValue}, x2 = {x2.varValue}, x3 = {x3.varValue} y x4 = {x4.varValue}, con valor optimo {value(problema.objective)}")

x1 = 0.0, x2 = 14.0, x3 = 0.0 y x4 = 1.0, con valor optimo 371.0


## Optimización sin restricciones.

## Librerías.

In [ ]:
import math
import sympy as sp
import numpy as np
import scipy as sci

## Funciones basicas.

In [15]:
def evaluar_func(funcion, values):
  #Evalua la funcion con los valores
  return eval(funcion, vars(math), values)

def derivate(funcion, var = ['x'], num = [1]):
  #recibe una funcion, una lista de variables con las que derivara y el numero de veces de cada una
  for i, v in enumerate(var):
    funcion = str(sp.diff(funcion, v, num[i]))
  return funcion

def find_in_list(target, py_list):
  #Encuentra target en py_list
  for i in range(len(py_list)):
    if py_list[i] == target:
      return i

### Optimización de una variable.

#### Método de sección dorada.
Es necesario darle a la función la funcion objetivo (`funcion`), un intervalo donde buscar (`cota_inf` y `cota_sup`) y por ultimo una tolerancia `h` que esta predefinida en $0.01$.

In [16]:
def sec_dorada(funcion, cota_inf, cota_sup, h = 0.01):
  lam = "a + (1 - (((5**(1 / 2)) - 1) / 2)) * (b - a)"
  miu = "a + ((((5**(1 / 2)) - 1) / 2) * (b - a))"

  row = [1, cota_inf, cota_sup, evaluar_func(lam, {"a":cota_inf, "b":cota_sup}), evaluar_func(miu, {"a":cota_inf, "b":cota_sup}), evaluar_func(funcion, {"x":evaluar_func(lam,{"a":cota_inf, "b":cota_sup})}), evaluar_func(funcion, {"x":evaluar_func(miu, {"a":cota_inf, "b":cota_sup})}), (cota_sup - cota_inf)]

  while row[7] >= h:
    #print(row)
    if row[5] >= row[6]:
      row[1] = row[3]
    else:
      row[2] = row[4]

    row[3] = evaluar_func(lam, {"a":row[1], "b":row[2]})
    row[4] = evaluar_func(miu, {"a":row[1], "b":row[2]})
    row[5] = evaluar_func(funcion, {"x":row[3]})
    row[6] = evaluar_func(funcion, {"x":row[4]})
    row[7] = row[2] - row[1]
    row[0] += 1

    opti = (row[1] + row[2]) / 2

  return (opti, evaluar_func(funcion, {'x':opti}))

In [19]:
print(f'el punto óptimo es: {sec_dorada("e**x+sin(x)", -4, 4, h = 0.0001)}')

el punto óptimo es: (-1.746147178353358, -0.8102206372964167)


In [20]:
print(f'el punto óptimo es: {sec_dorada("sin(x)*e**x", -2, 1, h = 0.001)}')

el punto óptimo es: (-0.7854243825048068, -0.32239694172320926)
